# Análisis clúster y análisis de anomalías

## 1. Introducción
Una de las tareas del análisis de datos consiste en la identificación de agrupaciones naturales de individuos.

Para esto, normalmente se cuenta con una base de datos cuyas fílas son los individuos de interés
(personas, empresas, transacciones, etc.) y las columnas son sus características. Esto último, que existan muchas variables por cada registro, es muy relevante en esta parte del curso y justifíca el uso de metodologías

multivariadas de agrupación. Estas técnicas usualmente se grupan bajo el título de análisis clúster.
Para el aprendizaje de máquina (machine learning) el análisis clúster hace parte del aprendizaje no
supervisado porque no se tienen concepciones a priori sobre cómo conformar los grupos.

## 2. Análisis clúster

El objetivo del análisis clúster es la identificación de grupos de individuos (o de variables) en conjuntos de datos multidimensionales. Los individuos (variables) que pertenecen a un mismo grupo deben ser parecidos, mientras que los grupos deben ser diferentes entre sí. Aquí, el concepto de parecido (o diferente) dependende de algún tipo de distancia entre las observaciones.
El análisis clúster es popular siempre que exista interes en grupos de individuos (variables). Por ejemplo, en mercadeo, es importante hacer segmentaciones de clientes de acuerdo con sus características para después aplicar estrategias de ventas diferentes enfocadas a cada subgrupo.
En la práctica, se deben tener en cuenta las siguientes recomendaciones:
* En los datos, los elementos que se quieren agrupar deben estar organizados por fílas, mientras que las caracterésticas que definen a estos elementos van por columnas.
* Se debe evitar la presencia de valores perdidos. Si existen se deben imputar o eliminar la observación, teniendo muy en cuenta las posibles consecuencias en términos de sesgo en los datos.
* Las variables deben tener escalas similares.

Con respecto al último punto, los algoritmos de clustering, en general, son sensibles a la escala de las variables. Por ejemplo, en una base de datos de países, se pueden tener como variables el producto interno bruto (PIB) y el crecimiento. Aquí, el PIB suele venir en unidades monetarias, y tener valores incluso de billones, mientras que el crecimiento es un porcentaje. De esta forma, la escala de las variables es muy diferente y el algoritmo tenderá a agrupar con base en la variable de mayor escala, en este caso el PIB. Para evitar esto, las variables pueden estandarizarse (llevar a series con media 0 y varianza 1) o reescalarse (todas tienen el mismo mínimo y máximo).

> **📝 Nota:** Los procesos de estandarización o reescalamiento no aplican si las variables que describen al individuo son categóricas. En este caso, se debe recurrir a opciones distintas como la construcción de variables dummys (dicotómicas, de ceros y unos) o utilizar medidas de distancia capaces de tratar con este tipo de información.

## 2.1. Medidas de distancia

La distancia entre dos observaciones mide su cercanía (como era de esperar). Para un grupo de individuos, se puede obtener una matriz que contenga la distancia entre todos ellos. Como resultado, la matriz de distancias es cuadrada (igual número de filas y columnas) y tiene ceros en su diagonal principal.

La distancia entre un par de individuos 1 y 2, definidos a partir de p variables $x_{1i}, x_{2i}, ... x_{pi}$ con $i = 1,2$ se puede calcular como:

* Distancia Euclídea: $d (x_1, x_2) = \sqrt{\sum_{j=1}^{p} (x_{j1}-x_{j2})^2}$.
* Distancia Manhattan (o del taxista):  $d (x_1, x_2) = \sum_{j=1}^{p} |x_{j1}-x_{j2}|$.
* Distancia de Mahalanobis: $d (x_1, x_2) = \sqrt{{(x_{j1}-x_{j2})^T} * {\Sigma}^{-1} (x_{1}-x_{2})}$ donde $\Sigma$ es la matriz de varianzas y covarianzas entre $x_1$ y $x_2$.

También existen otras medidas basadas en la correlacion entre individuos que son utilizadas en contextos
específicos. En este caso, se considera que dos individuos son parecidos si tienen una alta correlación en sus características, aún cuando los valores que éstas tomen sean muy diferentes:
* Distancia de correlación Pearson: $d (x_1, x_2) = 1 - \dfrac{\sum_{j=1}^{p} (x_{j1} - \bar x)(x_{j2} - \bar x)}{\sqrt{\sum_{j=1}^{p} (x_{j1} - \bar x)^2(x_{j2} - \bar x)^2}}$.

Muchas de las medidas de distancia aplican únicamente a variables numéricas, pero en la práctica es
usual que las características de los individuos estén definidas como series ordinales o incluso categoricas.

Para estos casos se ha desarrollado la distancia de Gower, que agrupa las variables de un conjunto de datos de acuerdo con su naturaleza y aplica las medidas de distancias pertinentes según cada caso: para variables numéricas y ordinales se utiliza la distancia Manhattan (con un ajuste por empates para el segundo caso), y para variables nominales con k categorías se generan k variables indicadoras (dicotómicas) y luego se aplican medidas de similaridad adecuadas. Las distancias de cada caso se combinan de manera lineal, usualmente como un promedio simple.

## 3. Implementando algoritmos en Python

### 3.1. Carga y exploración de datos
A continuación se utilizarán datos empresariales con informacion de estados financieros. En primera instancia, se cargan y exploran los datos, para esto se utilizará la libreria de Pandas:


In [1]:
import pandas as pd
